# Regularizacija 

Regularizacija je tehnika masinskog ucenja koja se koristi za kontrolu:
* preprilagodjavanja (overfitting) modela
* multikolinearnosti atributa
* kompleksnosti i interpretacije modela (nekad izbora atributa) 

<br>**I na taj nacin, u nekom smislu, kontrolise, odnosno popravlja model.**

### Linearna regresija sa regularizacijom

### Ridge regresija:

[Dokumentacija](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html)

Funkcija greske: 
$$L_{ridge}(\beta) = ||\beta X -y||^2 + \lambda ||\beta||^2$$

### LASSO regresija:

[Dokumentacija](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)

Funkcija greske: 
$$L_{lasso}(\beta) = ||\beta X -y||^2 + \lambda |\beta|$$

### Regularizacioni hiperparametar $\lambda$ kontrolise vrednosti koeficijenata $\beta$ - sto je vece $\lambda$ vise smo kaznili, odnosno smanjili koeficijente linearne regresije tj. redukovali kompleksnost modela, multikolinearnost i overfitting.

<img src='lasso.png'>

### Razlika: Samo u LASSO regresiji koeficijenti se mogu anulirati! <br> Zato LASSO regresiju mozemo koristiti za selekciju atributa, samim tim i redukciju dimenzije, dok Ridge regresiju ne mozemo.

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics
from sklearn import datasets

Koristicemo poznati Boston skup podataka za predikciju cena nekretnina (koji smo koristili za samu linearnu regresiju):

In [ ]:
data = datasets.load_boston()

In [ ]:
X = data.data
y = data.target

Kratka analiza podataka:

In [ ]:
data.feature_names

In [ ]:
number_of_features = len(data.feature_names)
number_of_features

In [ ]:
X = data.data # features

In [ ]:
X.shape

In [ ]:
y = data.target # target variable

In [ ]:
y.shape

In [ ]:
X = pd.DataFrame(data.data, columns=data.feature_names) # za dalji rad nam treba DataFrame
y = pd.DataFrame(data.target, columns=['MEDV'])

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
X.info() # all float columns and no missing data

Priprema za treniranje modela:

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.33, random_state = 23)

In [ ]:
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

### Linearna regresija

Prvo cemo napraviti bazni model linearne regresije kao i do sada:

In [ ]:
linear = linear_model.LinearRegression()

In [ ]:
linear.fit(X_train, y_train)

In [ ]:
linear.coef_

In [ ]:
for feature, coef in zip(data.feature_names, linear.coef_.ravel()):
    print (feature, coef)

In [ ]:
linear.intercept_

In [ ]:
y_pred = linear.predict(X_test)

In [ ]:
linear_test_score = linear.score(X_test, y_test) # koeficijent determinacije na test skupu

In [ ]:
linear_train_score = linear.score(X_train, y_train) # koeficijent determinacije na train skupu

In [ ]:
print('Ocena na skupu za treniranje: ', linear_train_score)
print('Ocena na skupu za testiranje: ', linear_test_score)

**Primetimo da postoji razlika u meri kvaliteta modela na testnom i trening skupu. Sto je veca razlika sa vise sigurnosti zakljucujemo da smo preprilagodili model trening podacima. Zato primenjujemo tehnike regularizacije koeficijenata.**

### Ridge regresija

Implementiramo model pomocu `scikit-learn` biblioteke preko klase `Ridge`. Ulogu regularizacionog hiperparametra $\lambda$ ima `alpha`.

In [ ]:
ridge = linear_model.Ridge(alpha = 10)

In [ ]:
ridge.fit(X_train, y_train)

In [ ]:
ridge.coef_

In [ ]:
for feature, coef in zip(data.feature_names, ridge.coef_.ravel()):
    print (feature, coef)

In [ ]:
ridge.intercept_  # slobodni koeficijent ne ulazi u regularizaciju

In [ ]:
ridge_train_score = ridge.score(X_train, y_train) 

In [ ]:
ridge_test_score = ridge.score(X_test, y_test) 

In [ ]:
print('Ocena na skupu za treniranje: ', ridge_train_score)
print('Ocena na skupu za testiranje: ', ridge_test_score)

### LASSO regresija

Implementiramo model pomocu `scikit-learn` biblioteke preko klase `Lasso`. Ulogu regularizacionog hiperparametra $\lambda$ ima `alpha`.

In [ ]:
lasso = linear_model.Lasso(alpha=1) # probati sa drugim vrednostima regularizacionog hiperparametra

In [ ]:
lasso.fit(X_train, y_train)

In [ ]:
lasso.coef_

In [ ]:
for feature, coef in zip(data.feature_names, lasso.coef_.ravel()):
    print (feature, coef)

In [ ]:
lasso.intercept_  # slobodni koeficijent ne ulazi u regularizaciju

In [ ]:
lasso_train_score = lasso.score(X_train, y_train) 

In [ ]:
lasso_test_score = lasso.score(X_test, y_test) 

In [ ]:
print('Ocena na skupu za treniranje: ', lasso_train_score)
print('Ocena na skupu za testiranje: ', lasso_test_score)

### Vizuelizacija koeficijenata modela

In [ ]:
plt.figure(figsize=(10, 5))
plt.xticks(np.arange(0, number_of_features), data.feature_names, rotation='90')
plt.plot(linear.coef_[0], '^', label='Linearna regresija' )
plt.plot(ridge.coef_[0], 'o', label='Ridge regresija (alpha=10)')
plt.plot(lasso.coef_, 'v', label='Lasso regresija (alpha=0.1)')
plt.plot(np.arange(0, number_of_features), np.zeros(number_of_features), color='gray', linestyle='--')
plt.legend(loc='best')
plt.show()